In [1]:
import plotly.graph_objs as go
from ipywidgets import Button, HBox, VBox, Output, HTML, Layout, Tab, IntText, FloatText, BoundedFloatText, BoundedIntText, Label
from IPython.display import display
from datetime import timedelta
from trading_algo import TradingAlgo, MovingAveragesParameter, BollingBandParameters
import logging
import threading
import time

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Global variable to store the trading algorithm instance
trading_algo = None

# Function to initialize the trading algorithm with user inputs
def initialize_trading_algo():
    global trading_algo
    
    moving_averages_params = MovingAveragesParameter(ma1.value, ma2.value, ma3.value)
    bollinger_bands_params = BollingBandParameters(bollinger_period.value, bollinger_deviation.value)
    
    trading_algo = TradingAlgo(
        file_location="C:\\Users\\juane\\Documents\\Python\\Baraktest\\etc\\trading_data.csv",
        moving_averages_params=moving_averages_params,
        bollinger_bands_params=bollinger_bands_params,
        initial_capital=initial_capital.value,
        max_risk=max_risk.value,
        limit_order_pct=limit_order_pct.value,
        millisec_execution_delay=timedelta(milliseconds=execution_delay.value),
        transaction_fees_per_contract=transaction_fees.value,
    )
    
    logger.info("Trading algorithm initialized with user parameters.")
    display_dashboard()

# Input fields for user parameters with grey background and right-aligned white description
def create_input_field(widget, description):
    # Create a white label for the input description
    label = HTML(f"<span style='color:white;'>{description}</span>")
    return HBox([label, widget])

ma1 = IntText(value=2, layout=Layout(width='150px', background_color='#555555', color='white'))
ma2 = IntText(value=5, layout=Layout(width='150px', background_color='#555555', color='white'))
ma3 = IntText(value=8, layout=Layout(width='150px', background_color='#555555', color='white'))
bollinger_period = IntText(value=13, layout=Layout(width='150px', background_color='#555555', color='white'))
bollinger_deviation = FloatText(value=3, layout=Layout(width='150px', background_color='#555555', color='white'))
initial_capital = IntText(value=1000, layout=Layout(width='150px', background_color='#555555', color='white'))
max_risk = BoundedFloatText(value=0.3, min=0, max=1, step=0.01, layout=Layout(width='150px', background_color='#555555', color='white'))
limit_order_pct = BoundedIntText(value=15, min=0, max=100, step=1, layout=Layout(width='150px', background_color='#555555', color='white'))
execution_delay = IntText(value=1000, layout=Layout(width='150px', background_color='#555555', color='white'))
transaction_fees = FloatText(value=0, layout=Layout(width='150px', background_color='#555555', color='white'))

# Start button to initialize the trading algorithm
start_button = Button(description="Start Trading", button_style='success', tooltip="Initialize Trading Algorithm")

# Function to display the dashboard
def display_dashboard():
    # Create Plotly figures
    def create_figure(title, xaxis_title, yaxis_title, line_color, marker_color, marker_symbol):
        fig = go.FigureWidget()
        fig.add_scatter(x=[], y=[], mode='lines+markers',
                        line=dict(color=line_color, width=2),
                        marker=dict(size=6, color=marker_color, symbol=marker_symbol))
        fig.update_layout(
            title=title,
            xaxis_title=xaxis_title,
            yaxis_title=yaxis_title,
            template="plotly_dark",
            margin=dict(l=40, r=20, t=50, b=40),
            height=400
        )
        return fig

    # Create figures
    price_fig = create_figure("Real-Time Price Data", "Time", "Price", '#1f77b4', '#ff7f0e', 'circle')
    pnl_fig = create_figure("Real-Time PnL Data (With Costs)", "Time", "PnL", '#2ca02c', '#d62728', 'diamond')
    pnl_no_cost_fig = create_figure("Real-Time PnL Data (Without Costs)", "Time", "PnL", '#9467bd', '#1f77b4', 'square')
    book_value_fig = create_figure("Real-Time Book Value", "Time", "Book Value", '#e377c2', '#ff9896', 'star')
    open_pos_fig = create_figure("Real-Time Open Positions", "Time", "Open Positions", '#d62728', '#7f7f7f', 'triangle-up')
    
    max_drawdown_fig = create_figure("Max Drawdown", "Time", "Max Drawdown", '#ff7f0e', '#2ca02c', 'hexagon')
    volatility_fig = create_figure("Volatility", "Time", "Volatility", '#17becf', '#bcbd22', 'cross')

    # Initialize output widgets
    log_output = Output()
    live_data_widget = HTML()
    unfilled_trades_output = HTML()
    last_trades_output = HTML()

    # Global controls
    auto_update = False
    batch_size = 500
    max_display_points = 1000

    # Function to update live data
    def update_live_data(time, open_position, price, pnl):
        live_data_widget.value = f"""
        <div style="font-family:Arial; color:lightgrey;">
        <h4>Live Data</h4>
        <b>Time:</b> {time}<br>
        <b>Open Position:</b> {open_position}<br>
        <b>Price:</b> {price:.2f}<br>
        <b>PnL:</b> {pnl:.2f}
        </div>
        """

    # Function to update the data arrays
    def update_data_arrays(unfilled_trades_df, last_trades_df):
        unfilled_trades_output.value = f"""
        <div style="font-family:Arial; color:lightgrey;">
        <h4>Unfilled Trades</h4>
        <pre>{unfilled_trades_df.to_string(index=False)}</pre>
        </div>
        """
        
        last_trades_output.value = f"""
        <div style="font-family:Arial; color:lightgrey;">
        <h4>Last Trades</h4>
        <pre>{last_trades_df.to_string(index=False)}</pre>
        </div>
        """

    # Function to update plots and data arrays
    def update_plots():
        with log_output:
            if auto_update:
                try:
                    times, prices, pnls, pnls_no_cost, book_values, open_positions, max_drawdowns, volatilities = [], [], [], [], [], [], [], []

                    for _ in range(batch_size):
                        has_data, dashboard_data = trading_algo.get_new_data()

                        if not has_data:
                            logger.debug("No more data available")
                            break

                        times.append(dashboard_data.current_time)
                        prices.append(dashboard_data.current_price)
                        pnls.append(dashboard_data.current_pnl)
                        pnls_no_cost.append(dashboard_data.pnl_exc_trading_costs)
                        book_values.append(dashboard_data.current_book_value)
                        open_positions.append(dashboard_data.open_pos)
                        max_drawdowns.append(dashboard_data.current_max_drawdown)
                        volatilities.append(dashboard_data.current_volatility)

                    if times:
                        update_live_data(times[-1], open_positions[-1], prices[-1], pnls[-1])

                        update_figure(price_fig, times, prices)
                        update_figure(pnl_fig, times, pnls)
                        update_figure(pnl_no_cost_fig, times, pnls_no_cost)
                        update_figure(book_value_fig, times, book_values)
                        update_figure(open_pos_fig, times, open_positions)
                        update_figure(max_drawdown_fig, times, max_drawdowns)
                        update_figure(volatility_fig, times, volatilities)

                        update_data_arrays(dashboard_data.current_unfilled_trades, dashboard_data.df_last_trades)

                        logger.debug(f"Batch updated with {len(times)} data points. Current display count: {len(price_fig.data[0].x)}")
                except Exception as e:
                    logger.error(f"Error during update: {e}")

    def update_figure(fig, x_data, y_data):
        fig.data[0].x += tuple(x_data)
        fig.data[0].y += tuple(y_data)
        if len(fig.data[0].x) > max_display_points:
            fig.data[0].x = fig.data[0].x[-max_display_points:]
            fig.data[0].y = fig.data[0].y[-max_display_points:]

    # Thread function for automatic updates
    def auto_update_thread():
        while auto_update:
            update_plots()
            time.sleep(1)

    # Button to start/stop the updates
    start_button = Button(description="Start", button_style='success', tooltip="Start automatic updates")
    stop_button = Button(description="Stop", button_style='danger', tooltip="Stop automatic updates")

    def start_auto_update(_):
        nonlocal auto_update
        auto_update = True
        logger.info("Started automatic updates")
        threading.Thread(target=auto_update_thread).start()

    def stop_auto_update(_):
        nonlocal auto_update
        auto_update = False
        logger.info("Stopped automatic updates")

    start_button.on_click(start_auto_update)
    stop_button.on_click(stop_auto_update)

    # Dashboard sections
    live_data_section = VBox([HTML("<h4 style='color:lightgrey;'>Live Data</h4>"), live_data_widget, price_fig],
                             layout=Layout(border='solid 1px lightgrey', padding='10px', background_color='#333333'))

    trade_history_section = VBox([HTML("<h4 style='color:lightgrey;'>Trade History</h4>"), last_trades_output],
                                 layout=Layout(border='solid 1px lightgrey', padding='10px', background_color='#333333'))

    positions_orders_section = VBox([HTML("<h4 style='color:lightgrey;'>Positions and Orders</h4>"), open_pos_fig, unfilled_trades_output],
                                    layout=Layout(border='solid 1px lightgrey', padding='10px', background_color='#333333'))

    pnl_section = VBox([HTML("<h4 style='color:lightgrey;'>PnL</h4>"), pnl_fig, pnl_no_cost_fig, book_value_fig],
                       layout=Layout(border='solid 1px lightgrey', padding='10px', background_color='#333333'))
    
    risk_section = VBox([HTML("<h4 style='color:lightgrey;'>Risk Metrics</h4>"), max_drawdown_fig, volatility_fig],
                       layout=Layout(border='solid 1px lightgrey', padding='10px', background_color='#333333'))

    # Organize the layout with some spacing and alignment
    button_box = HBox([start_button, stop_button], layout=Layout(justify_content='center', padding='10px'))
    tabs = Tab([live_data_section, trade_history_section, positions_orders_section, pnl_section, risk_section])
    tabs.set_title(0, 'Live Data')
    tabs.set_title(1, 'Trade History')
    tabs.set_title(2, 'Positions and Orders')
    tabs.set_title(3, 'PnL')
    tabs.set_title(4, 'Risk')

    dashboard_layout = VBox([button_box, tabs, log_output], layout=Layout(background_color='#222222', padding='10px'))

    # Display the dashboard
    display(dashboard_layout)

# Apply custom CSS for dashboard styling
custom_css = """
<style>
    body {
        background-color: #222222;
        color: #f0f0f0;
        font-family: Arial, sans-serif;
    }
    .widget-tab, .widget-box {
        background-color: #333333;
        border: 1px solid #444444;
        color: #f0f0f0;
    }
    .widget-button {
        background-color: #444444;
        color: #f0f0f0;
        border: 1px solid #555555;
    }
    .widget-html-content h4 {
        color: #f0f0f0;
    }
    .table-dark {
        background-color: #333333;
        color: #f0f0f0;
    }
    .table-dark th {
        background-color: #444444;
        color: #f0f0f0;
    }
    .table-dark td {
        background-color: #333333;
        color: #f0f0f0;
    }
</style>
"""
display(HTML(custom_css))

# Set up front page with input fields
input_layout = VBox([
    HTML("<h3 style='color:lightgrey;'>Enter Trading Algorithm Parameters</h3>"),
    create_input_field(ma1, 'MA1:'),
    create_input_field(ma2, 'MA2:'),
    create_input_field(ma3, 'MA3:'),
    create_input_field(bollinger_period, 'Bollinger Period:'),
    create_input_field(bollinger_deviation, 'Bollinger Deviation:'),
    create_input_field(initial_capital, 'Initial Capital:'),
    create_input_field(max_risk, 'Max Risk:'),
    create_input_field(limit_order_pct, 'Limit Order %:'),
    create_input_field(execution_delay, 'Execution Delay (ms):'),
    create_input_field(transaction_fees, 'Transaction Fees:'),
    start_button
], layout=Layout(border='solid 1px lightgrey', padding='10px', background_color='#333333'))

# Display the input layout
display(input_layout)

# Bind start button click to initialize function
start_button.on_click(lambda _: initialize_trading_algo())




HTML(value='\n<style>\n    body {\n        background-color: #222222;\n        color: #f0f0f0;\n        font-f…

INFO:__main__:Trading algorithm initialized with user parameters.


INFO:__main__:Started automatic updates
INFO:__main__:Stopped automatic updates
